In [ ]:
import random
import time
import math

from pathlib import Path
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import ipywidgets as ipw

from IPython.display import clear_output, display, Markdown

In [ ]:
#plot the charts right into your Jupyter Notebook.
# %matplotlib notebook 

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

<IPython.core.display.Javascript object>

In [ ]:
out_fetch = ipw.Output(layout=ipw.Layout(width = '1000px', border='solid'))
out_read = ipw.Output(layout=ipw.Layout(width = '1000px', border='solid'))
out_average = ipw.Output(layout=ipw.Layout(width = '1000px', border='solid'))
out_plot = ipw.Output(layout=ipw.Layout(width = '1000px', border='solid'))
out_histo = ipw.Output(layout=ipw.Layout(width = '1000px', border='solid'))

# List of usable fields
listOfFields = []
workingField = ["All"]

# Datafiles lists
fileTypes = ["Data" , "LAMMPS" , "openMM" , "CSV"]
listOfFiles = []
localFileName = "__working_file__"
listOfFilesLocal = []
workingFile = ["None"]

dataUsed = [0, 100]

def clearDataFields():
    listOfFields.clear()
    workingField.clear()
    workingField.append("All")
    
def clearFilesRead():
    listOfFiles.clear()
    listOfFilesLocal.clear()
    workingFile.clear()
    workingFile.append("None")

def averageClean(btn):
    with out_average:
        out_average.clear_output()

def makePlotClean(btn):
    with out_plot:
        out_plot.clear_output()

def makeHistoClean(btn):
    with out_histo:
        out_histo.clear_output()

def reset(btn):
    try:
        del data
    except:
        pass
    
    # Clear all screen output
    with out_average:
        out_average.clear_output()

    with out_plot:
        out_plot.clear_output()

    with out_fetch:
        out_fetch.clear_output()

    with out_read:
        out_read.clear_output()

    try:
        for f in listOfFilesLocal:
            if os.path.exists(f):
                os.remove(f)
    except:
        pass

    clearDataFields()
    clearFilesRead()
    


In [ ]:
def fetch(btn):
    with out_fetch:
        out_fetch.clear_output()

    with out_read:
        out_read.clear_output()

    with out_plot:
        out_plot.clear_output()

    with out_average:
        out_average.clear_output()

    n = 0
    for file in fileName.value.split():
        flocal = localFileName + str(n)
        with out_fetch:
            print(file)
            command = "scp " + file + " " + flocal
            error = os.system(command)
        if error == 0:
            listOfFiles.append(file)
            listOfFilesLocal.append(flocal)
            n = n + 1
        else:
            msg = "ERROR - failed fetching file : "+file
            print(msg)

    if n > 0:
        createMenuFiles(listOfFiles)
        
        # --- Add read button ---
        btn_read = ipw.Button(description="Read", layout=ipw.Layout(width="150px"))
        btn_read.on_click(read)
        btn_readClear = ipw.Button(description="Clear data", layout=ipw.Layout(width="150px"))
        btn_readClear.on_click(readClear)

        with out_fetch:
            r1 = ipw.HBox([btn_read,btn_readClear])
            r2 = ipw.HBox([out_read])
            r3 = ipw.VBox([r1,r2])
            display(r3)


In [ ]:
def readClear(btn):
    with out_read:
        out_read.clear_output()

def read(btn):
    global data

    try:
        clearDataFields()
    except:
        pass

    with out_read:
        out_read.clear_output()

    with out_plot:
        out_plot.clear_output()

    with out_average:
        out_average.clear_output()

    if activefileType.value == "Data":
        data = readDataFile(workingFile[0])

    elif activefileType.value == "LAMMPS":
        data = readDataLammps(workingFile[0])

    elif activefileType.value == "openMM":
        data = readDataOpenMM(workingFile[0])

    elif activefileType.value == "CSV":
        data = pd.read_csv(workingFile[0], sep=",")
        header = []
        for c in data.columns:
            header.append(c)
    
    with out_read:
        print("Number of data points read    :",len(data))
        print("------------------------------")

        print("{:>31s}".format("Fields (Min/Max) :",len(data.columns)))
        for i in range(0,len(data.columns)):
            if "Progress" in data.columns[i] or \
               "Remaining" in data.columns[i]:
                continue

            f = data.columns[i]
            m1 = np.min(data[f].apply(pd.to_numeric))
            m2 = np.max(data[f].apply(pd.to_numeric))
            print("{:>31s} {:15.5f} {:15.5f}".format(f+" :",float(m1),float(m2)))
#             print(m1,m2)
        print("------------------------------")
        with out_read:
            createMenulistOfFields(listOfFields)
            
        n1 = len(data)
        dataUsed[0] = ipw.IntRangeSlider(value=[1,n1], min=1, max=n1, 
                                         layout=ipw.Layout(width='90%'))        
        s1 = ipw.HBox([ipw.Label(value="Data point range", layout=ipw.Layout(width="150px")), dataUsed[0]])
        display(s1)

######## compute averages
        btn_average = ipw.Button(description="Average", layout=ipw.Layout(width="150px"))
        btn_average.on_click(average)
        btn_averageClean = ipw.Button(description="Clean Average", layout=ipw.Layout(width="150px"))
        btn_averageClean.on_click(averageClean)

######## plot vs time
        btn_plot = ipw.Button(description="Plot", layout=ipw.Layout(width="150px"))
        btn_plot.on_click(makePlot)
        btn_plotClean = ipw.Button(description="Clean Plot", layout=ipw.Layout(width="150px"))
        btn_plotClean.on_click(makePlotClean)

######## histograms
        btn_histo = ipw.Button(description="Histogram", layout=ipw.Layout(width="150px"))
        btn_histo.on_click(makeHisto)
        btn_histoClean = ipw.Button(description="Clean Histogram", layout=ipw.Layout(width="150px"))
        btn_histoClean.on_click(makeHistoClean)

######## display all buttons        
        display(ipw.HBox([btn_average,btn_plot,btn_histo]))
        display(ipw.HBox([btn_averageClean,btn_plotClean,btn_histoClean]))

######## display output boxes
        outputBoxes = []
        outputBoxes.append(ipw.HBox([out_average]))
        outputBoxes.append(ipw.HBox([out_plot]))
        outputBoxes.append(ipw.HBox([out_histo]))
        display(ipw.VBox(outputBoxes))


In [ ]:
def readDataFile(localFILE):

    lines = []
    with open(localFILE, 'r') as file:
        for line in file:
            if "#" in line:
                continue
            line = line.strip() # removes spaces
            lines.append(line.split())

        data = pd.DataFrame(lines)

    header = ["X"]
    for c in range(1,len(data.columns)):
        header.append("Y"+str(c))
    data.columns = header
    
    for f in header:
        if f == "X":
            continue
        listOfFields.append(f)
    
    return data

In [ ]:
def readDataOpenMM(localFILE):
    data = pd.read_csv(localFILE, sep=" ", comment="#")

    openMMfile = open(localFILE, 'r')
    firstLine = openMMfile.readline()
    openMMfile.close()

    firstLine = firstLine.replace("#",'')
    firstLine = firstLine.replace('" "','@')
    firstLine = firstLine.replace('"','')
    firstLine = firstLine.replace('\n','')

    header = firstLine.split('@')
    data.columns = header

    for f in header:
        if f in ("Progress (%)", "Step" , "Time (ps)", "Elapsed Time (s)" , "Time Remaining"):
            continue
        listOfFields.append(f)

    return data

In [ ]:
def readDataLammps(localFILE):

    iBlock = 0
    lines = []
    with open(localFILE, 'r') as file:
        for line in file:
            line = line.strip() # removes spaces

            if "Step" in line:
                iBlock = abs(iBlock) + 1
                header = line.split()
                continue
            elif "Loop" in line:
                iBlock = -iBlock

            elif "WARNING" in line:
                continue

            if iBlock > 0:
                values = line.split()
                lines.append(values)

        data = pd.DataFrame(lines)
        
    data.columns = header

    for f in header:
        if f in ("Step", "Time", "CPU", "CPULeft"):
            continue
        listOfFields.append(f)
    
    return data

In [10]:
def dropdown_filesHandler(change):
    for i in range(0,len(listOfFiles)):
        if listOfFiles[i] == change.new:
            workingFile[0] = listOfFilesLocal[i]

def createMenuFiles(data):
    global activefileType
    if len(data) == 1:
        option_list = data
        workingFile[0] = localFileName+"0"
    else:
        option_list = ["Choose one"]
        option_list.extend(data)

    dropdown = ipw.Dropdown(description="Active file:", options=option_list, layout={'width':'auto'})
    dropdown.observe(dropdown_filesHandler, names='value')

    activefileType = ipw.RadioButtons(options=fileTypes,value='LAMMPS')

    with out_fetch:
        display(dropdown)
        display(activefileType)

def dropdown_listOfFieldsHandler(change):
    if change.new == "All":
        workingField[0] = "All"
    else:
        for i in range(0,len(listOfFields)):
            if listOfFields[i] == change.new:
                workingField[0] = listOfFields[i]
            
def createMenulistOfFields(data):
    global activeField
    option_list = ["All"]
    option_list.extend(data)

    dropdown = ipw.Dropdown(description="Active field:", options=option_list, layout={'width':'auto'})
    dropdown.observe(dropdown_listOfFieldsHandler, names='value')

    with out_read:
        display(dropdown)


In [11]:
def average(btn):
    with out_average:
        out_average.clear_output()

    val = dataUsed[0].value
    i0 = val[0] - 1
    i1 = val[1] - 1

    with out_average:
        print("First point for average        :" , i0+1)
        print("Last point for average         :" , i1+1)
        print("------------------------------")

    for i in range(0,len(data.columns)):
        if "Progress" in data.columns[i] or \
           "Remaining" in data.columns[i]:
            continue
        
        f = data.columns[i]
        if (f in workingField) or workingField[0] == "All" :
            y = data[f].apply(pd.to_numeric)

            avg = np.mean(y[i0:i1])
            stdev = np.std(y[i0:i1])
            stder = np.std(y[i0:i1]) / np.sqrt(i1-i0+1)

            with out_average:
                print(f)
                string = "Average - StdErr - Stdev      :"
                print("{:31s} {:15.5f} {:15.5f} {:15.5f}".format(string, avg, stder, stdev))
                print("------------------------------")

In [12]:
def makePlot(btn):
    with out_plot:
        out_plot.clear_output()

    if activefileType.value == "LAMMPS":
        xlabel = "Time (ps)"
        x = data["Time"].apply(pd.to_numeric)

    elif activefileType.value == "openMM":
        xlabel = "Time (ps)"
        x = data["Time (ps)"].apply(pd.to_numeric)
        
    else:
        xlabel = "X"
        x = data["X"].apply(pd.to_numeric)

    val = dataUsed[0].value
    i0 = val[0] - 1
    i1 = val[1] - 1
  
    with out_plot:
        print("Start time for plotting       :" , x[i0])
        print("Final time for plotting       :" , x[i1])
        print("------------------------------")
        print(" ")

    for f in listOfFields:
        if (f in workingField) or ("All" in workingField) :            
            y = data[f].apply(pd.to_numeric)

            fig = plt.figure(figsize=(30, 5))
            ax = fig.gca()

            fs = 24
            ax.plot(x[i0:i1],y[i0:i1])
            plt.xlabel(xlabel, fontsize=fs)
            plt.xticks(fontsize=fs)
            plt.ylabel(f, fontsize=fs)
            plt.yticks(fontsize=fs)

            if activefileType.value == "LAMMPS" or activefileType.value == "openMM":
                fit = np.polyfit(x[i0:i1],y[i0:i1],1)
                drift = np.poly1d(fit)
                xdrift = range(int(x[i0]),int(x[i1]))
                ydrift = drift(xdrift)
                ax.plot(xdrift,ydrift,c='red')

                lw = 3
                ax.spines["top"].set_linewidth(lw)
                ax.spines["bottom"].set_linewidth(lw)
                ax.spines["left"].set_linewidth(lw)
                ax.spines["right"].set_linewidth(lw)

            with out_plot:
                plt.show()

In [13]:
# Define model function to be used to fit to the data above:
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

def makeHisto(btn):
    with out_histo:
        out_histo.clear_output()

    val = dataUsed[0].value
    i0 = val[0] - 1
    i1 = val[1] - 1

    with out_histo:
        print("First point for histo        :" , i0+1)
        print("Last point for histo         :" , i1+1)
        print("------------------------------")

    for i in range(0,len(data.columns)):
        if "Progress" in data.columns[i] or \
           "Remaining" in data.columns[i]:
            continue
        
        f = data.columns[i]
        if (f in workingField) or workingField[0] == "All" :
            y = data[f].apply(pd.to_numeric)

            # 
            hist, bin_edges = np.histogram(y, density=True, bins=50)
            bin_centres = (bin_edges[:-1] + bin_edges[1:])/2
            # p0 is the initial guess for the fitting coefficients (A, mu and sigma above)
            avg = np.mean(y[i0:i1])
            stdev = np.std(y[i0:i1])
            p0 = [1., avg, stdev]

            coeff, var_matrix = curve_fit(gauss, bin_centres, hist, p0=p0)
            # Get the fitted curve
            hist_fit = gauss(bin_centres, *coeff)

            fig = plt.figure(figsize=(30, 5))
            ax = fig.gca()
#             bins = np.linspace(bound_min, bound_max, 50)
            ax.plot(bin_centres, hist, label='Test data')
            ax.plot(bin_centres, hist_fit, label='Fitted data')
            
            with out_histo:
                # Finally, lets get the fitting parameters, i.e. the mean and standard deviation:
                print('Fitted mean = ', coeff[1])
                print('Fitted standard deviation = ', coeff[2])
                print("------------------------------")
                plt.show()


In [14]:
actions = []
# --- Input files ---
fileName = ipw.Textarea(value='/Users/237454K/Research/new_2022_forcefield/so4_ti/nosym/log.sym',
                    placeholder='Enter LAMMPS outputfile location here',
                    description='',
                    rows=3,                  
                    layout=ipw.Layout(width="auto"))
actions.append(fileName)

# --- Fetch files button ---
btn_fetch = ipw.Button(description="Fetch", layout=ipw.Layout(width="150px"))
btn_fetch.on_click(fetch)

# --- Clear all button ---
btn_reset = ipw.Button(description="Clear all", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)
reset(btn_reset)

# --- Display buttons ---
actions.append(ipw.HBox([btn_fetch,btn_reset]))
actions.append(ipw.HBox([out_fetch]))

ipw.VBox(actions)
